<a href="https://colab.research.google.com/github/RogerHeederer/ForAllPytorch/blob/main/LongSeq_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
torch.manual_seed(0)

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [6]:
#문장에서 중복된 character 제거 후 단일 세트 만들기
char_set = list(set(sentence))
char_set

['b',
 'u',
 'l',
 'y',
 's',
 ' ',
 ',',
 'e',
 'f',
 'k',
 'r',
 'd',
 'a',
 'c',
 '.',
 'o',
 'i',
 'h',
 "'",
 'n',
 't',
 'm',
 'p',
 'w',
 'g']

In [8]:
#단어 집합 만들기
char_dic = {c: i for i, c in enumerate(char_set)}
char_dic

{' ': 5,
 "'": 18,
 ',': 6,
 '.': 14,
 'a': 12,
 'b': 0,
 'c': 13,
 'd': 11,
 'e': 7,
 'f': 8,
 'g': 24,
 'h': 17,
 'i': 16,
 'k': 9,
 'l': 2,
 'm': 21,
 'n': 19,
 'o': 15,
 'p': 22,
 'r': 10,
 's': 4,
 't': 20,
 'u': 1,
 'w': 23,
 'y': 3}

In [9]:
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10
learning_rate = 0.1

In [10]:
dic_size

25

데이터 세팅 부분

In [12]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i+1: i + sequence_length + 1]
  print(i, x_str, '->', y_str)

  #정수 인코딩 수행 : char를 char_dic에서 만든 인덱스로 변형하여 정수 값으로 넣기 
  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [40]:
x_data[1]

[8, 5, 3, 15, 1, 5, 23, 12, 19, 20]

In [14]:
# 정답 데이터의 정수 값을 보면 1칸씩 밀려 있는 값들임을 확인
y_data[1]

[5, 3, 15, 1, 5, 23, 12, 19, 20, 5]

In [16]:
np.eye(dic_size)[3]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
# 정수 인코딩 값을 원핫인코딩에 반영
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

In [20]:
x_one_hot[1] #인덱스에 위 정수값을 반영 [8, 5, 3, 15, 1, 6, 23, 12, 19, 20]

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

# 데이터 X의 shape 변천사
**(170,10) 형태의 텍스트 배열 --> (170,10) 형태의 정수인코딩 배열 --> (170,10,25) 형태의 원핫 인코딩 배열 --> (170,10,25) 형태의 파이토치 텐서**

# 데이터 Y의 shape 변천사
**(170,10) 형태의 텍스트 배열 --> (170,10) 형태의 정수인코딩 배열 --> (170,10) 형태의 파이토치 텐서**

In [ ]:
# 데이터 X의 shape 변천사
# (170,10) 형태의 텍스트 배열 --> (170,10) 형태의 정수인코딩 배열 --> 170,10,25 원핫 인코딩 배열 --> 170,10,25 형태의 파이토치 텐서


In [21]:
# 파이토치 텐스로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [67]:
X.shape, Y.shape

(torch.Size([170, 10, 25]), torch.Size([170, 10]))

In [22]:
# RNN + FC를 가진 클래스 정의
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    
    #rnn 정의 / batch_first=True를 설정하면, 배치사이즈 차원이 제일 앞에 위치한 shape이 됨
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    #Fully Connected Layer 정의
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [23]:
rnn_network = Net(dic_size, hidden_size, 2) # rnn 2층으로 쌓겠다

In [24]:
rnn_network

Net(
  (rnn): RNN(25, 25, num_layers=2, batch_first=True)
  (fc): Linear(in_features=25, out_features=25, bias=True)
)

모델을 만들었으니 임의의 값으로 테스트 먼저

In [32]:
X.size() # 배치사이즈 ,시퀀스길이, input차원

torch.Size([170, 10, 25])

In [81]:
#위의 X 데이터 shape을 참조하여 임의 값 생성
value = torch.rand(170, 10, 25)
value[1] # 문장 길이 10. 문장을 구성하는 글자의 차원은 25

tensor([[0.6319, 0.1413, 0.9725, 0.0826, 0.5798, 0.9984, 0.1246, 0.6208, 0.2113,
         0.0807, 0.4378, 0.3538, 0.6501, 0.7687, 0.1639, 0.2726, 0.6752, 0.4234,
         0.8159, 0.0748, 0.7863, 0.7280, 0.7366, 0.5348, 0.9089],
        [0.9183, 0.9811, 0.0466, 0.0482, 0.7039, 0.1205, 0.2445, 0.4557, 0.7406,
         0.5935, 0.1859, 0.8012, 0.4547, 0.4156, 0.6645, 0.1763, 0.3860, 0.8427,
         0.2473, 0.6174, 0.3026, 0.1309, 0.1469, 0.8903, 0.1831],
        [0.2724, 0.8042, 0.0177, 0.8872, 0.6853, 0.2934, 0.5483, 0.9031, 0.2997,
         0.9385, 0.4509, 0.7898, 0.1136, 0.7522, 0.3219, 0.9387, 0.5594, 0.2351,
         0.1547, 0.3037, 0.3920, 0.8514, 0.4068, 0.8223, 0.0439],
        [0.3545, 0.5143, 0.8789, 0.8336, 0.3832, 0.9821, 0.7542, 0.8686, 0.5618,
         0.7861, 0.5153, 0.8900, 0.6593, 0.5697, 0.6471, 0.3746, 0.4890, 0.3080,
         0.2154, 0.3816, 0.5307, 0.7355, 0.3218, 0.6167, 0.8126],
        [0.4535, 0.5754, 0.5059, 0.0666, 0.5856, 0.4105, 0.8526, 0.1180, 0.5760,
       

In [82]:
test_result = rnn_network(value)

In [83]:
test_result

tensor([[[-0.1816,  0.1516,  0.1394,  ..., -0.1032, -0.1000, -0.0405],
         [-0.1374,  0.0852,  0.3739,  ..., -0.0565, -0.2179,  0.0639],
         [-0.0991,  0.1674,  0.2009,  ...,  0.0045, -0.2253,  0.1084],
         ...,
         [-0.0606,  0.1541,  0.2712,  ..., -0.0177, -0.2871,  0.1366],
         [-0.1072,  0.1249,  0.2799,  ..., -0.0640, -0.1948,  0.1612],
         [-0.0179,  0.1779,  0.2790,  ..., -0.0841, -0.1108,  0.2271]],

        [[-0.1575,  0.1724,  0.1162,  ...,  0.0073, -0.1330,  0.0377],
         [-0.0283,  0.0463,  0.2468,  ...,  0.0121, -0.2710,  0.0362],
         [-0.1886,  0.1132,  0.2927,  ..., -0.1286, -0.2644,  0.0482],
         ...,
         [-0.0596,  0.1222,  0.2078,  ...,  0.0847, -0.3082,  0.1075],
         [-0.1312,  0.2438,  0.2759,  ...,  0.0074, -0.2029,  0.1387],
         [-0.0204,  0.1308,  0.2714,  ...,  0.0716, -0.2846,  0.1519]],

        [[-0.2415,  0.1713,  0.1982,  ...,  0.0033, -0.1817,  0.0085],
         [-0.0517,  0.1317,  0.1492,  ...,  0

In [84]:
test_result.shape

torch.Size([170, 10, 25])

In [85]:
test_result.argmax(dim=2)

tensor([[13,  7, 13,  ..., 13,  7,  7],
        [ 7, 13, 13,  ..., 13,  7,  7],
        [ 7, 13, 13,  ...,  7, 13,  7],
        ...,
        [ 7,  7, 13,  ..., 13,  7, 13],
        [13, 13, 13,  ...,  7,  7,  7],
        [ 7,  7, 13,  ...,  7,  7,  7]])

In [91]:
# 170개의 문장이 있음. 각 문장의 길이는 10임. 문장을 구성하는 하나의 글자 차원은 25임
# argmax(dim=2)는 가장 안쪽 차원들 값 중 가장 큰 확률을 가진 값을 리턴하라
# 즉 한 글자를 이루는 25차원 값들 중 가장 큰 값 하나만 남김
# [[0.11,0.13,0.7,0.001.......0.03],[..25개..],[..25개..],[],[],[],[],[],[],[]]
# 위 25개 차원 수 중에서 가장 큰 확률을 가지는 원소를 하나 뽑아서, 그 놈의 인덱스를 넣어줌
# 가령 이렇게 되는 거임 [3, 1, 2, 14, 12, 7, 7, 9, 23, 24]
# 그럼 남게 되는 거는, 170개의 문장. 각 문장의 길이는 10.
test_result.argmax(dim=2).shape

torch.Size([170, 10])

# 데이터 X의 shape 변천사
**(170,10) 형태의 텍스트 배열 --> (170,10) 형태의 정수인코딩 배열 --> (170,10,25) 형태의 원핫 인코딩 배열 --> (170,10,25) 형태의 파이토치 텐서**

# 데이터 Y의 shape 변천사
**(170,10) 형태의 텍스트 배열 --> (170,10) 형태의 정수인코딩 배열 --> (170,10) 형태의 파이토치 텐서**

In [94]:
test_result.view(-1, dic_size).shape, Y.view(-1).shape
#FC 레이어에 넣어주기 위해서 일자로 펴는 Flatten 작업
#기존 (170, 10, 25) 형태를 -> (170*10, 25) 형태로 변환

#한 글자에 25차원으로 이루어진 1700개 글자를, 1차원 정수 값으로 이루어진 1700개 글자와 비교하여 손실값을 계산하는 방식임

(torch.Size([1700, 25]), torch.Size([1700]))

In [26]:
_## 손실 함수 및 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss() #멀티 카테고리를 분류할 때 사용하는 손실 함수
optimizer = optim.Adam(rnn_network.parameters(), learning_rate)

In [97]:
#훈련 시작
for i in range(100):
  optimizer.zero_grad()
  outputs = rnn_network(X)
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) # 손실 값 측정
  loss.backward() #backpropagation 수행
  optimizer.step() #최적의 파라미터를 찾아가며 업데이트 함

In [100]:
results = outputs.argmax(dim=2)
results, results.shape

(tensor([[ 2,  5,  3,  ..., 12, 19, 20],
         [ 5,  3, 15,  ..., 19, 20,  5],
         [20, 15,  1,  ..., 20,  5, 20],
         ...,
         [15, 15,  8,  ...,  5,  4,  7],
         [20,  8,  5,  ...,  4,  7, 12],
         [19,  5, 20,  ...,  7, 12, 13]]), torch.Size([170, 10]))

In [99]:
predict_str = ""
for j, result in enumerate(results):
    # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
    if j == 0:
        predict_str += ''.join([char_set[t] for t in result])
    else:
        predict_str += char_set[result[-1]]

print(predict_str)

l you want to build a ship, don't arum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the seac
